In [70]:
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps
from scipy.sparse import csr_matrix, find
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [71]:
#nltk.download()

In [72]:
COLUMN_AXIS = 1
CONTENT_COLNAME = 'conteudo'
SUBTITLE_COLNAME = 'subTitulo'
TITLE_COLNAME = 'titulo'

In [73]:
news = pd.read_csv("../data/estadao_noticias_eleicao.csv", encoding="utf-8")

In [74]:
empty_str = ""
news.titulo.fillna(empty_str, inplace=True)
news.subTitulo.fillna(empty_str, inplace=True)
news.conteudo.fillna(empty_str, inplace=True)

In [75]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo

In [76]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

In [77]:
stopword_ = stopwords.words('portuguese') + ['os']
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

# 1

In [78]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [79]:
tokens_lists = content.apply(lambda text: text.lower().split())

In [80]:
tokens = [token for tokens_list in tokens_lists for token in tokens_list]

In [81]:
matrix, vocab_to_index = co_occurrence_matrix(tokens)

In [82]:
consultable_matrix = matrix.tocsr()

In [83]:
def consult_frequency(consultable_matrix, vocab_to_index, w1, w2):
    return(consultable_matrix[vocab_to_index[w1], vocab_to_index[w2]])

In [84]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(consultable_matrix, vocab_to_index, w1, w2)

3

# 2

In [85]:
index_to_vocab = {v: k for k, v in vocab_to_index.items()}

In [86]:
#indi, indj, indv = find(consultable_matrix)
#three_dim_matrix = list(zip(indi, indj, indv))
three_dim_matrix = list(zip(
    matrix.row, matrix.col, matrix.data))

In [87]:
def get_freq(mtuple):
    return mtuple[2]

def get_top3(word, three_dim_matrix):
    w_index = vocab_to_index[w1]
    freqs = list(filter(lambda mt: mt[0] == w_index, three_dim_matrix))
    max_freq = sorted(freqs, key=get_freq, reverse=True)
    sorted_indexes_by_freq = list(map(lambda mtuple: mtuple[1], max_freq))
    top3_index = sorted_indexes_by_freq[:3]
    top3_words = list(map(lambda i: index_to_vocab[i], top3_index))
    return top3_words

In [88]:
print(get_top3('poucos', three_dim_matrix))

['dias', 'os', 'meses']


# 3

True